<img style="float: right; margin-right: 30px;" src="https://cscw.acm.org/2016/images/logo@257px.png">

The following is a brief example of using [topic modeling](https://en.wikipedia.org/wiki/Topic_model) within a [Jupyter](http://jupyter.org) notebook. Jupyter is a web based programming and publishing environment that works with over 40 different programming languages.

Since Jonathan and Karen went to [CSCW 2016](https://cscw.acm.org/2016/) a few weeks ago I thought it might be fun to try to use topic modeling to try to characterize the papers that were submitted there.

I downloaded the PDFs for all 142 papers and converted them to text. Since the formatting was fairly structured I was also able to extract the abstracts from the text. You can see both the paper text and the abstracts in the [data](data) directory.

I then wrote a bit of helper code using Python's [Gensim](https://radimrehurek.com/gensim/) topic modeling library to (hopefully) illustrate a little bit of how topic modeling works. The first thing we need to do is to import some of this helper code.

In [1]:
from topicmodel import papers, abstracts, topics

The `papers` and `abstracts` functions are Python generators that return each word from each of the CSCW papers and abstracts. So for example we can see the words in the first abstract by calling the `abstracts` function and calling `next` on the result:

In [2]:
next(abstracts())

['world',
 'becomes',
 'more',
 'digitized',
 'interconnected',
 'ormation',
 'that',
 'once',
 'considered',
 'private',
 'such',
 'health',
 'status',
 'being',
 'shared',
 'publicly',
 'understand',
 'this',
 'phenomenon',
 'better',
 'crucial',
 'study',
 'what',
 'types',
 'health',
 'information',
 'being',
 'shared',
 'social',
 'media',
 'well',
 'whom',
 'this',
 'paper',
 'study',
 'traits',
 'users',
 'share',
 'their',
 'personal',
 'health',
 'ﬁtness',
 'related',
 'inform',
 'social',
 'media',
 'analyzing',
 'ﬁtness',
 'status',
 'updates',
 'that',
 'MyFitnessPal',
 'users',
 'have',
 'shared',
 'Twitter',
 'investigate',
 'certain',
 'features',
 'like',
 'user',
 'proﬁle',
 'ﬁtness',
 'activity',
 'network',
 'social',
 'media',
 'potentially',
 'impact',
 'engagement',
 'ﬁtness',
 'users',
 'also',
 'discuss',
 'impl',
 'ations',
 'ﬁndings',
 'achieve',
 'better',
 'retention',
 'these',
 'users',
 'promote',
 'more',
 'sharing',
 'their',
 'status',
 'updates']

As you can see the text isn't pefect, but it's largely good enough for these purposes. We can then generate a topic model using the `topics` function. `topics` will create a [Latent Dirichlet Allocation](https://www.quora.com/What-is-a-good-explanation-of-Latent-Dirichlet-Allocation) model and then use the [Umass Topic Coherence](http://ciir-publications.cs.umass.edu/getpdf.php?id=956) algorithm to list the primary topics. I'll be the first to admit that I have little to no idea what that means. It's what the Gensim documentation told me. Perhaps Philip Resnick will have gone over some of this terminology beforehand.

In [3]:
topics(abstracts)

1. that, their, with, social, online
2. that, with, from, work, data
3. with, that, social, media, their
4. that, with, social, users, technology
5. with, that, this, conflict, more


The results are not very interesting because we mostly just see the most commonly ocurring words in most English text. However we can create a list (or really a Python set) of words to ignore when doing the modeling.

In [4]:
words = set(["the", "of", "to", "a"])

And then we can call the `topics` function again, but this time passing in our list of words to ignore, which are usuall called *stop words*:

In [5]:
topics(abstracts, ignore=words)

1. that, social, their, with, this
2. that, with, work, from, this
3. that, with, social, more, this
4. that, with, their, social, online
5. that, with, technology, this, work


But as you can see there are a lot more words that would be good to ignore. Luckily other people have run into this issue before and compiled lists of these extremely common English words, and I've included them here so we can import them.

In [6]:
from stopwords import stopwords

Here's how many words are in the list:

In [7]:
len(stopwords)

319

So now we can try running `topics` again with this longer list of words to ignore:

In [8]:
topics(abstracts, ignore=stopwords)

1. social, work, people, study, support
2. social, study, content, design, data
3. social, users, study, data, work
4. social, work, online, users, media
5. data, social, online, technology, media


Now finally things are getting a little bit more interesting! 

One important thing to note about LDA is that it is a *generative model* which uses *randomness* as part of the algorithm. So if we run `topics` again with the exact same options it will generate different results. How does this impact the way you might use topic modeling?

In [9]:
topics(abstracts, ignore=stopwords)

1. media, social, study, work, design
2. social, data, study, work, users
3. social, work, users, study, content
4. data, online, privacy, paper, design
5. social, people, online, self, design


On inspection it looks like there words that are used a lot in CSCW papers that might be useful to add to our ignore list:

In [22]:
stopwords.add("social")
stopwords.add("work")
stopwords.add("study")
stopwords.add("paper")
stopwords.add("data")
stopwords.add("online")
stopwords.add("design")
stopwords.add("technology")
stopwords.add("users")
stopwords.add("media")
stopwords.add("people")
stopwords.add("results")
stopwords.add("content")
stopwords.add("information")
stopwords.add("systems")

In [23]:
topics(abstracts, ignore=stopwords)

1. friends, inﬂuence, Likes, quality, provide
2. support, health, present, research, mobile
3. community, communities, present, network, interaction
4. support, MOOCs, findings, sharing, video
5. different, analysis, time, privacy, task


At this point it might be useful to try to assign labels to some of the topic groups:

1. **social media**: friends, inﬂuence, Likes, quality, provide
2. **health**: support, health, present, research, mobile
3. **community**: community, communities, present, network, interaction
4. **online learning**: support, MOOCs, findings, sharing, video
5. **privacy**: different, analysis, time, privacy, task

One very important thing to know about LDA topic modeling is that it is a generative statistical technique: it uses *randomness* as part of the algorithm. So if we run our `topics` helper function again with the exact same options we will get different results:

In [12]:
topics(abstracts, ignore=stopwords)

1. children, food, parents, porters, community
2. privacy, support, different, interviews, time
3. group, community, communities, networks, present
4. strategies, systems, research, sharing, collaborative
5. learning, trust, present, feedback, monitoring


How does this randomness impact how you can use topic modeling as a tool in different problem domains?

The `topics` helper function has some additional knobs you can turn to change the output. For example you can change the number of topics you would like to see:

In [13]:
topics(abstracts, ignore=stopwords, num_topics=10)

1. infrastructural, SAHDs, interface, sensemaking, decay
2. children, feedback, personal, present, parents
3. trust, friends, inﬂuence, network, teams
4. conflict, participation, porters, apps, vendors
5. food, analysis, interaction, systems, user
6. privacy, mobile, research, Likes, present
7. health, privacy, video, news, approach
8. community, trust, practice, based, monitoring
9. collaborative, network, cultural, identify, analytics
10. support, strategies, group, students, MOOCs


And you can change the number of words in each topic:

In [14]:
topics(abstracts, ignore=stopwords, num_words=3)

1. support, children, present
2. trust, research, strategies
3. task, collaborative, provide
4. mobile, food, networks
5. group, privacy, Instagram


Remember the `papers` generator we imported at the beginning? Well that contains all the text of the paper. Here's the text of the first paper:

In [15]:
next(papers())

['CSCW',
 'FEBRUARY',
 'MARCH2',
 '2016',
 'FRANCISCO',
 'Persistent',
 'Sharing',
 'Fitness',
 'Status',
 'Twitter',
 'Kunwoo',
 'Park',
 'KAIST',
 'South',
 'Korea',
 'park',
 'kaist',
 'Meeyoung',
 'KAIST',
 'South',
 'Korea',
 'meeyoungcha',
 'kaist',
 'Ingmar',
 'Weber',
 'QCRI',
 'Qatar',
 'iweber',
 'Chul',
 'MyFitnessPal',
 'United',
 'States',
 'clee',
 'myﬁtnesspal',
 'ABSTRACT',
 'world',
 'becomes',
 'more',
 'digitized',
 'interconnected',
 'formation',
 'that',
 'once',
 'considered',
 'private',
 'such',
 'health',
 'status',
 'being',
 'shared',
 'publicly',
 'understand',
 'this',
 'phenomenon',
 'better',
 'crucial',
 'study',
 'what',
 'types',
 'health',
 'information',
 'being',
 'shared',
 'social',
 'media',
 'well',
 'whom',
 'this',
 'paper',
 'study',
 'traits',
 'users',
 'share',
 'their',
 'personal',
 'health',
 'ﬁtness',
 'related',
 'informa',
 'tion',
 'social',
 'media',
 'analyzing',
 'ﬁtness',
 'status',
 'updates',
 'that',
 'MyFitnessPal',
 'users'

Let's run the papers through the LDA topic model:

In [16]:
topics(papers, ignore=stopwords)

1. mobile, privacy, trust, different, activity
2. practice, different, strategies, crowd, case
3. present, provide, support, community, feedback
4. task, sharing, privacy, support, strategies
5. support, group, parents, MOOCs, children


Does looking at the fulltext of the paper change the modeling at all? Try playing around with the code if you want by adding stopwords, or changing the number of topics or words returned. 

That's all folks!